In [12]:
import json
import os
from pathlib import Path
from typing import Any

import requests
import requests_cache
from loguru import logger

requests_cache.install_cache("data/mal_cache", backend="sqlite", expire_after=86400)

In [13]:
BASE_DIR = Path(os.path.abspath("")).resolve().parent
RAW_DIR = BASE_DIR / "data" / "raw"
META_DIR = BASE_DIR / "data" / "metadata"
SUMMARY_DIR = BASE_DIR / "data" / "summaries"

META_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)


JIKAN_BASE = "https://api.jikan.moe/v4"

In [28]:
def save_data(url: str, data: dict[str, Any]) -> None:
    # url example: DIR / f"{process_query}_{page}.json"
    if not data:
        return
    with open(url, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)  # Handle Japanese


def fetch_metadata_from_myanimelist(query: str) -> list[dict[str, Any]]:
    logger.info(f"[+] Searching MAL: {query:6>}")
    resp = requests.get(f"{JIKAN_BASE}/anime", params={"q": query, "limit": 20})
    resp.raise_for_status()
    result = resp.json()

    # TODO: Check for pagination
    pagination = result["pagination"]
    page = 1
    process_query = query.replace(" ", "-").replace("/", "_")
    with open(RAW_DIR / f"{process_query}_{page}.json", "w", encoding="utf-8") as f:
        json.dump(result, f, indent=2, ensure_ascii=False)
    animes_data = result["data"]
    return animes_data


def filter_anime_metadata(animes_data: dict[str, Any]) -> dict[str, Any]:
    animes_data = [
        r
        for r in animes_data
        if r["type"].lower() in {"tv", "movie", "ova", "special", "tv_special"}
    ]
    return animes_data


def fetch_episodes(mal_id: int) -> list[dict[str, Any]]:
    episodes = []
    page = 1
    while True:
        logger.info(f"[+] Searching MAL Episodes: {mal_id:6} - Page {page:2}")
        url = f"{JIKAN_BASE}/anime/{mal_id}/episodes?page={page}"
        resp = requests.get(url)
        resp.raise_for_status()
        data = resp.json()
        if not data.get("data"):
            break
        episodes.extend(data["data"])
        if not data.get("pagination", {}).get("has_next_page"):
            break
        page += 1
    return episodes

In [29]:
animes_data = fetch_metadata_from_myanimelist("Kaguya Sama Love is War!")
animes_data = filter_anime_metadata(animes_data)

2025-07-08 10:27:27.053 | INFO     | __main__:fetch_metadata_from_myanimelist:10 - [+] Searching MAL: Kaguya Sama Love is War!


In [ ]:
for anime in animes_data:
    mal_id = anime["mal_id"]
    episodes = fetch_episodes(mal_id)
    data = {"summary": anime, "episodes": episodes}
    save_data(url=META_DIR / f"{mal_id}.json", data=data)

2025-07-08 10:27:27.197 | INFO     | __main__:fetch_episodes:39 - [+] Searching MAL Episodes:  37999 - Page  1
2025-07-08 10:27:27.260 | INFO     | __main__:fetch_episodes:39 - [+] Searching MAL Episodes:  40591 - Page  1
2025-07-08 10:27:27.264 | INFO     | __main__:fetch_episodes:39 - [+] Searching MAL Episodes:  43608 - Page  1
2025-07-08 10:27:27.267 | INFO     | __main__:fetch_episodes:39 - [+] Searching MAL Episodes:  52198 - Page  1
2025-07-08 10:27:27.270 | INFO     | __main__:fetch_episodes:39 - [+] Searching MAL Episodes:  43609 - Page  1
2025-07-08 10:27:27.272 | INFO     | __main__:fetch_episodes:39 - [+] Searching MAL Episodes:  23229 - Page  1
2025-07-08 10:27:27.274 | INFO     | __main__:fetch_episodes:39 - [+] Searching MAL Episodes:   3545 - Page  1
2025-07-08 10:27:27.276 | INFO     | __main__:fetch_episodes:39 - [+] Searching MAL Episodes:   3545 - Page  2
2025-07-08 10:27:27.278 | INFO     | __main__:fetch_episodes:39 - [+] Searching MAL Episodes:   3545 - Page  3
2